Student id: 24206493

### Task 1: Data Collection

Choose one of the four data sources listed here:
http://mlg.ucd.ie/modules/python/assignment1
In your first notebook, apply web scraping in Python to collect and parse all of the data from your chosen source. 
Save the collected data in an appropriate format for subsequent analysis.

For this assigment, I am using the Blue Motive Cars website that provides listings for second-hand car sales in Ireland on this link: http://mlg.ucd.ie/modules/python/assignment1/cars/index.html. I start by importing urllib and beautifulSoup libraries to fetch and read the response from the url link provided and then to parse the HTML response respectively.


In [13]:
import urllib
import bs4

url = "http://mlg.ucd.ie/modules/python/assignment1/cars/index.html"
response = urllib.request.urlopen(url)
bluecars = response.read().decode()

Create a BeautifulSoup object which contains the structure of the HTML page using HTML parser feature

In [14]:
soup = bs4.BeautifulSoup(bluecars, 'html.parser') 
print(soup)

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="noindex" name="robots"/>
<meta content="Content on this site is posted for teaching purposes only. Original data is from theguardian.com" name="description"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Car Sale Records</title>
<link href="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" target="_blank"/>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.3/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/js/bootstrap.min.js"></script>
<link href="style.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<div class="container mtb">
<div class="logo">
<a href="index.html"><img alt="Property Price Register" class="logo" src="logo.png"/></a>
</div>
<div class="row instructions">
<div class="col-md-12" id="content">
<p ;="" style="font-s

Create an explicit function, getlinks() that takes BS obj as prameter and does link gathering from the current page whenever called. This will be important later when we will have to gather the 20 odd links for one cartype.

In [15]:
def getlinks(BSobject):
    links = BSobject.find_all("a")
    baseurl = "http://mlg.ucd.ie/modules/python/assignment1/cars/"
    list = ""
    list1 = []
    for link in links:
        car_html = link.get("href")
        list = baseurl + car_html
        list1.append(list)
    list1.pop(0)
    return list1

The gettext() function takes 2 parameters, the extracted "li" list and a dummy list for storing the output dictionaries.
I created this function in hindsight when I saw that this part of getting text from the "li" list was getting duplicated mainly to keep the code crisp. 
Additionally, I have added text.strip to avoid wasting time on cleaning the text feature-wise for the analysis. This was added after I started analysis in Notebook 2 and got caught up with spaces around these values. 

In [16]:
def gettext(modelname, temp_list):
    for j in modelname:
        dict_car = {}
        Make = j.find("span", {"class" : "make-model"}).text.strip()    # Make-model is not in a <td> tag but in a separate span class in the <li> tag so the treatment is different for it
        dict_car["Make-Model"] = Make
        td_list = j.find_all("td")
        for i, td in enumerate(td_list):
            if td.text == 'Date of Sale:':
                dict_car["Date of Sale"] = td_list[i+1].text.strip() 
            if td.text == 'Sale Price:':
                dict_car["Sale Price"] = td_list[i+1].text.strip() 
            if td.text == 'Year:':
                dict_car["Year"] = td_list[i+1].text.strip() 
            if td.text == 'Mileage:':
                dict_car["Mileage"] = td_list[i+1].text.strip() 
            if td.text == 'Classification:':
                dict_car["Classification"] = td_list[i+1].text.strip() 
            if td.text == 'Transmission:':
                dict_car["Transmission"] = td_list[i+1].text.strip() 
            if td.text == 'Fuel Type:':
                dict_car["Fuel Type"] = td_list[i+1].text.strip() 
            if td.text == 'Description:':
                dict_car["Description"] = td_list[i+1].text.strip() 
            if td.text == 'Sale Location:':
                dict_car["Sale Location"] = td_list[i+1].text.strip() 
        temp_list.append(dict_car)       
    return temp_list 

#### The list "main_list" has the links of 4 car-types fetched from the BlueCars main webpage using getlinks() function. My strategy is three-fold: 

1. Use getlinks() to get all the links on pages
2. Use gettext() to get all the car model details on the page and put it in a dictionary which goes into a list. 
For each cartype, there will be 4 lists of dictionaries in the final list (final_retriever).
3. Export the text in a CSV file using csv.DictWriter for further data preparation and analysis

In [17]:
main_list = getlinks(soup)
final_retriever = []  # this is our main guy here. It will have all the 4 lists of dictionaries

for mainlink in main_list:
    response = urllib.request.urlopen(mainlink)
    Cars = response.read().decode()
    cartype_soup = bs4.BeautifulSoup(Cars, 'html.parser')
    #print(cartype_soup) # to check if we got the html data
    list2 = getlinks(cartype_soup)
    list2.pop(-1)    # this list is crucial as it gets all the links from the first page itself. It saves time as we will see in our next for loop.
    #print(list2)    #check
    list = []
    modelname = cartype_soup.find_all("li")  # modelname has all the <li> tags as a list in it and the <td> tag that has all the relavent data for us is a child of the <li> tag.
    list = gettext(modelname, list)
    for j in list2:
        response = urllib.request.urlopen(j)
        Cars = response.read().decode()
        cartype_soup = bs4.BeautifulSoup(Cars, 'html.parser')
        modelname = cartype_soup.find_all("li")
        list = gettext(modelname, list)
        #print(list)   #check
    final_retriever.append(list)

#checking if we got all the data
print(len(final_retriever[0]), len(final_retriever[1]), len(final_retriever[2]), len(final_retriever[3]))
    


384 396 593 327




CSV library is imported to create and write into a new csv file for writing all the data fetched in the list, final_retriever under relavent headers. I use two for loops, one for accessing the first list that has one type of car-type and then another one to access the dictionaries of that car-type.



In [18]:
import csv
fout = open('final_retriever.csv', 'w')
fields = ['Make-Model', 'Date of Sale', 'Sale Price', 'Year', 'Mileage', 'Classification', 'Transmission', 'Fuel Type', 'Description', 'Sale Location' ]
writer = csv.DictWriter(fout, fieldnames = fields)
writer.writeheader()
for x in final_retriever:
    for m in x:
        writer.writerow(m)

fout.close()